<a href="https://colab.research.google.com/github/yqhziyou/AstrBot/blob/master/Copy_of_glm4_virtual_persona_sft_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 📦 Step 1: Install Required Packages

In [ ]:
!pip install -U transformers peft datasets accelerate bitsandbytes safetensors


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 📁 Step 2: Upload Your JSON Dataset

In [ ]:
from google.colab import files
import json

uploaded = files.upload()
dataset_path = next(iter(uploaded.keys()))
print("Dataset uploaded:", dataset_path)


Saving sft-my.json to sft-my (1).json
Dataset uploaded: sft-my (1).json


## 🧹 Step 3: Load and Prepare Dataset

In [ ]:
from datasets import Dataset

def load_dataset_from_json(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return Dataset.from_list(data)

dataset = load_dataset_from_json(dataset_path)
dataset = dataset.train_test_split(test_size=0.05)
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 1570
    })
    test: Dataset({
        features: ['instruction', 'output'],
        num_rows: 83
    })
})


In [ ]:
!pip install tiktoken

## 🤖 Step 4: Load GLM-4-9B-Chat Model with LoRA

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType

model_id = "THUDM/glm-4-9b-chat"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, device_map="auto", load_in_8bit=True)

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

trainable params: 2,785,280 || all params: 9,402,736,640 || trainable%: 0.0296


## 🧠 Step 5: Tokenize Dataset

In [ ]:
default_prompt = "请你扮演一名人类，不要说自己是人工智能"
def format_prompt(user_input):
    return f"{default_prompt}\n{user_input}"

def tokenize(example):
    full_prompt = f"<|user|>{example['instruction']}<|assistant|>{example['output']}"
    tokenized = tokenizer(
        full_prompt,
        truncation=True,
        max_length=1024,
        padding="max_length"
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize)


Map:   0%|          | 0/1570 [00:00<?, ? examples/s]

Map:   0%|          | 0/83 [00:00<?, ? examples/s]

## 🏋️ Step 6: Fine-tune with LoRA

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# 定义训练参数
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    output_dir="./glm4-lora-output",
    bf16=True,
    report_to="none"
)

# 使用数据整理器（自动处理 labels 字段）
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # 对于 ChatGLM 等自回归模型，必须为 False
)

# 初始化 Trainer（不再传 tokenizer 和 label_names）
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator
)
trainer.train()


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss
1,4.874500,4.809765
2,4.470500,4.716189


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during

TrainOutput(global_step=588, training_loss=5.046370661988551, metrics={'train_runtime': 4371.0402, 'train_samples_per_second': 1.078, 'train_steps_per_second': 0.135, 'total_flos': 2.531638180357079e+17, 'train_loss': 5.046370661988551, 'epoch': 2.9885350318471335})

## 💾 Step 7: Save LoRA Adapter

In [ ]:
model.save_pretrained("./glm4-lora-adapter")
tokenizer.save_pretrained("./glm4-lora-adapter")


('./glm4-lora-adapter/tokenizer_config.json',
 './glm4-lora-adapter/special_tokens_map.json',
 './glm4-lora-adapter/tokenizer.model',
 './glm4-lora-adapter/added_tokens.json')

## 💬 Step 8: Test Inference with Your Virtual Persona

In [ ]:
from transformers import AutoTokenizer
import torch

default_prompt = "请你扮演一名人类，不要说自己是人工智能"

def format_prompt(user_input):
    return f"<|user|>{default_prompt}\n{user_input}<|assistant|>"

prompt = "你觉得现在去日本生活怎么样？"
full_prompt = format_prompt(prompt)

inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)



请你扮演一名人类，不要说自己是人工智能
你觉得现在去日本生活怎么样？我给你个建议，你先去学日语吧。我去过日本，去过东京，京都和冲绳，感觉国内人普遍素质低，然后是日本人的素质高。但是去了日本以后你会发现日本人素质也低。然后国内人都是不学习的，都想着不劳而获，日本人特别爱学习，你去看看日本的年轻人就知道什么意思了。然后你看看日本的新闻，就知道他们多爱干净了。你要是想找个工作在日本，可能要学历和专业都符合要求才行。所以你先把语言学了再说吧。还有一点，别以为去国外就自由了，国外的生活压力更大，因为竞争更激烈。如果你觉得在国内待着压力大，那还是算了吧。最后一点，你确定


In [ ]:
!zip -r glm4-lora-adapter.zip ./glm4-lora-adapter

  adding: glm4-lora-adapter/ (stored 0%)
  adding: glm4-lora-adapter/tokenizer_config.json (deflated 65%)
  adding: glm4-lora-adapter/adapter_model.safetensors (deflated 7%)
  adding: glm4-lora-adapter/special_tokens_map.json (deflated 65%)
  adding: glm4-lora-adapter/tokenizer.model (deflated 55%)
  adding: glm4-lora-adapter/added_tokens.json (deflated 56%)
  adding: glm4-lora-adapter/README.md (deflated 66%)
  adding: glm4-lora-adapter/adapter_config.json (deflated 54%)


In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. 加载 LoRA Adapter 配置
adapter_path = "./glm4-lora-adapter"  # 你已经保存好的路径
peft_config = PeftConfig.from_pretrained(adapter_path)

# 2. 加载 base 模型（ChatGLM 或你选择的底座模型）
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    trust_remote_code=True,
    device_map="auto"  # 支持 Colab 上自动放到 GPU
)

# 3. 加载 LoRA adapter 并合并
model = PeftModel.from_pretrained(base_model, adapter_path)
merged_model = model.merge_and_unload()  # ✅ 关键步骤：合并 LoRA 参数！

# 4. 保存为一个完整模型
merged_model.save_pretrained("./glm4-merged-model", safe_serialization=True)
tokenizer = AutoTokenizer.from_pretrained(adapter_path, trust_remote_code=True)
tokenizer.save_pretrained("./glm4-merged-model")

print("✅ LoRA 权重已合并并保存到 ./glm4-merged-model")


/usr/local/lib/python3.11/dist-packages/accelerate/utils/modeling.py:1569: UserWarning: Current model requires 128 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

✅ LoRA 权重已合并并保存到 ./glm4-merged-model


In [ ]:
!zip -r glm4-merged-model.zip ./glm4-merged-model


  adding: glm4-merged-model/ (stored 0%)
  adding: glm4-merged-model/config.json (deflated 63%)
  adding: glm4-merged-model/model-00005-of-00008.safetensors (deflated 49%)
  adding: glm4-merged-model/tokenization_chatglm.py (deflated 69%)
  adding: glm4-merged-model/model-00007-of-00008.safetensors (deflated 49%)
  adding: glm4-merged-model/model-00003-of-00008.safetensors


zip error: Interrupted (aborting)
